# Liriscat paper experiments
### 1. Init
#### 1.1. Import libraries (necessary)

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["PYTHONHASHSEED"] = "0"
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":4096:8"

import liriscat
liriscat.utils.set_seed(0)

import logging
import gc
import json
import torch
liriscat.utils.set_seed(0)
import pandas as pd

#### 1.2. Set up the loggers (recommended)

In [2]:
liriscat.utils.setuplogger(verbose = True, log_name="liriscat", debug=False)

### 2. CDM prediction
#### 2.1. Training and testing, sequential version

In [3]:
import warnings

gc.collect()
torch.cuda.empty_cache()

In [4]:
config = liriscat.utils.generate_eval_config(load_params=True,CDM='ncdm', esc = 'error', valid_metric= 'rmse', pred_metrics = ["mi_acc"], profile_metrics = ['meta_doa'], save_params=False, n_query=6, num_epochs=100, batch_size=512)
liriscat.utils.set_seed(config["seed"])

config["dataset_name"] = "algebra"
logging.info(config["dataset_name"])
config['learning_rate'] = 0.0001
config['inner_user_lr'] = 0.0016969685554352153
config['lambda'] = 2.2656270501845414e-06

config['meta_lr'] = 0.5
config['learning_users_emb_lr'] = 0.0005
config['patience'] = 20#5
config['num_inner_users_epochs'] = 3
config['d_in'] = 4
config['num_responses'] = 12


#pred_metrics,df_interp = test(config)

CUDA is available. Using GPU.
[INFO 33:05] algebra


In [5]:
def pareto_index2(d) : 
    d_acc = d[0]
    d_meta = d[1]

    r = []

    for i in range(len(d_acc)):
        r.append((0.5-d_acc[i]['mi_acc'])*(0.5-d_meta[i]['meta_doa']))
    return sum(r)

In [6]:
logging.info(f'#### {config["dataset_name"]} ####')
logging.info(f'#### config : {config} ####')
config['embs_path']='../embs/'
config['params_path']='../ckpt/'

gc.collect()
torch.cuda.empty_cache()

# Dataset downloading for doa and rm
warnings.filterwarnings("ignore", message="invalid value encountered in divide")
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Concept map format : {question_id : [category_id1, category_id2, ...]}
concept_map = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_concept_map.json', 'r'))
concept_map = {int(k): [int(x) for x in v] for k, v in concept_map.items()}

## Metadata map format : {"num_user_id": ..., "num_item_id": ..., "num_dimension_id": ...}
metadata = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_metadata.json', 'r'))


## Tensor containing the nb of modalities per question
nb_modalities = torch.load(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_nb_modalities.pkl',weights_only=True)


[INFO 33:05] #### algebra ####
[INFO 33:05] #### config : {'seed': 0, 'dataset_name': 'algebra', 'load_params': True, 'save_params': False, 'embs_path': '../embs/', 'params_path': '../ckpt/', 'early_stopping': True, 'esc': 'error', 'verbose_early_stopping': False, 'disable_tqdm': False, 'valid_metric': 'rmse', 'learning_rate': 0.0001, 'batch_size': 512, 'valid_batch_size': 10000, 'num_epochs': 100, 'eval_freq': 1, 'patience': 20, 'device': device(type='cuda'), 'lambda': 2.2656270501845414e-06, 'tensorboard': False, 'flush_freq': True, 'pred_metrics': ['mi_acc'], 'profile_metrics': ['meta_doa'], 'num_responses': 12, 'low_mem': False, 'n_query': 6, 'CDM': 'ncdm', 'i_fold': 0, 'num_inner_users_epochs': 3, 'num_inner_epochs': 10, 'inner_lr': 0.0001, 'inner_user_lr': 0.0016969685554352153, 'meta_lr': 0.5, 'meta_trainer': 'Adam', 'num_workers': 0, 'pin_memory': False, 'debug': False, 'learning_users_emb_lr': 0.0005, 'd_in': 4} ####


[INFO 38:14] #### config : {'seed': 0, 'dataset_name': 'math2', 'load_params': True, 'save_params': False, 'embs_path': '../embs/', 'params_path': '../ckpt/', 'early_stopping': True, 'esc': 'error', 'verbose_early_stopping': False, 'disable_tqdm': False, 'valid_metric': 'mi_acc', 'learning_rate': 0.0001, 'batch_size': 512, 'valid_batch_size': 10000, 'num_epochs': 100, 'eval_freq': 1, 'patience': 20, 'device': device(type='cuda'), 'lambda': 9.972254466547545e-06, 'tensorboard': False, 'flush_freq': True, 'pred_metrics': ['mi_acc'], 'profile_metrics': ['meta_doa'], 'num_responses': 12, 'low_mem': False, 'n_query': 6, 'CDM': 'impact', 'i_fold': 0, 'num_inner_users_epochs': 3, 'num_inner_epochs': 10, 'inner_lr': 0.0001, 'inner_user_lr': 0.016848380924625605, 'meta_lr': 0.5, 'meta_trainer': 'Adam', 'num_workers': 0, 'pin_memory': False, 'debug': False, 'learning_users_emb_lr': 0.001, 'd_in': 4} ####


In [7]:
# NCDM
meta_trainers = ['GAP']
for meta_trainer in meta_trainers : 
    config['meta_trainer'] = meta_trainer
    logging.info(f'#### meta_trainer : {config["meta_trainer"]} ####')
    for i_fold in range(1) : 
        config['i_fold'] = i_fold
            
        logging.info(f'#### i_fold : {i_fold} ####')
        ## Dataframe columns : (user_id, question_id, response, category_id)
        train_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_train_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})
        valid_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_valid_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})
        test_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_test_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})

        train_data = liriscat.dataset.CATDataset(train_df, concept_map, metadata, config,nb_modalities)
        valid_data = liriscat.dataset.EvalDataset(valid_df, concept_map, metadata, config,nb_modalities)
        test_data = liriscat.dataset.EvalDataset(test_df, concept_map, metadata, config,nb_modalities)

        for seed in range(1) :
            config['seed'] = seed
            logging.info(f'#### seed : {seed} ####')

            train_data.reset_rng()
            valid_data.reset_rng()
            test_data.reset_rng()

            S = liriscat.selectionStrategy.Random(metadata,**config)
            S.init_models(train_data, valid_data)
            S.train(train_data, valid_data)
            liriscat.utils.set_seed(0)
            S.reset_rng()
            d = (S.evaluate_test(test_data, train_data, valid_data))
            logging.info(d)
            logging.info(liriscat.utils.pareto_index(d))
        torch.cuda.empty_cache()

[INFO 33:06] #### meta_trainer : GAP ####
[INFO 33:06] #### i_fold : 0 ####
[INFO 33:33] #### seed : 0 ####
[INFO 33:33] Random_cont_model
../ckpt/algebra_NCDM_fold_0_seed_0
[INFO 33:34] compiling selection model
[INFO 33:38] ------- START Training
[INFO 33:38] train on cuda


/lustre/fsn1/projects/rech/enh/unv34ei/liriscat/liriscat/selectionStrategy/random.py:30: UserWarning: get_params() Notimplemented
  warnings.warn('get_params() Notimplemented')
  0%|          | 0/100 [00:00<?, ?it/s]

[INFO 33:40] - meta_params: 0.5
[INFO 33:40] - cross_cond: 0.5
[INFO 33:40] - meta_lambda: 0.5
[INFO 33:40] - learning_users_emb: 0.0005
[INFO 33:40] rmse : 0.4891076982021332
[INFO 33:40] valid_rmse : 2.946431875228882
[INFO 33:40] valid_loss : 5.026264667510986


  1%|          | 1/100 [00:02<04:18,  2.61s/it]

[INFO 33:42] - meta_params: 0.5
[INFO 33:42] - cross_cond: 0.5
[INFO 33:42] - meta_lambda: 0.5
[INFO 33:42] - learning_users_emb: 0.0005
[INFO 33:43] rmse : 0.48902466893196106
[INFO 33:43] valid_rmse : 2.9459316730499268
[INFO 33:43] valid_loss : 5.021945476531982


...

[INFO 35:33] - meta_params: 0.0001220703125
[INFO 35:33] - cross_cond: 0.0001220703125
[INFO 35:33] - meta_lambda: 0.0001220703125
[INFO 35:33] - learning_users_emb: 1.220703125e-07
[INFO 35:33] rmse : 0.486935555934906
[INFO 35:33] valid_rmse : 2.933346748352051
[INFO 35:33] valid_loss : 4.914449214935303


 55%|█████▌    | 55/100 [01:55<01:34,  2.09s/it]

[INFO 35:33] -- END Training --
[INFO 35:33] train on cuda



100%|██████████| 6/6 [00:00<00:00,  9.47it/s]


[INFO 35:57] ({0: {'mi_acc': 0.69744473695755}, 1: {'mi_acc': 0.69744473695755}, 2: {'mi_acc': 0.69744473695755}, 3: {'mi_acc': 0.69744473695755}, 4: {'mi_acc': 0.69744473695755}, 5: {'mi_acc': 0.6972299814224243}}, {0: {'meta_doa': np.float64(0.486973732475573)}, 1: {'meta_doa': np.float64(0.4880515469740171)}, 2: {'meta_doa': np.float64(0.489487092037853)}, 3: {'meta_doa': np.float64(0.4890770466088384)}, 4: {'meta_doa': np.float64(0.49200824003568105)}, 5: {'meta_doa': np.float64(0.4890502338331308)}})
[INFO 35:57] -0.01290107825768087


In [9]:
config['meta_trainer'] = 'MAML'

for seed in range(1) :
    config['seed'] = seed
    logging.info(f'#### seed : {seed} ####')
    
    train_data.reset_rng()
    valid_data.reset_rng()
    test_data.reset_rng()


    S = liriscat.selectionStrategy.Random(metadata,**config)
    S.init_models(train_data, valid_data)
    S.train(train_data, valid_data)
    liriscat.utils.set_seed(0)
    S.reset_rng()
    d = (S.evaluate_test(test_data, train_data, valid_data))
    print(liriscat.utils.pareto_index(d))
    print(d)
    torch.cuda.empty_cache()
    del S

[INFO 37:11] #### seed : 0 ####
[INFO 37:11] Random_cont_model
../ckpt/algebra_NCDM_fold_0_seed_0
[INFO 37:12] compiling selection model
[INFO 37:12] ------- START Training
[INFO 37:12] train on cuda


/lustre/fsn1/projects/rech/enh/unv34ei/liriscat/liriscat/selectionStrategy/random.py:30: UserWarning: get_params() Notimplemented
  warnings.warn('get_params() Notimplemented')
  0%|          | 0/100 [00:00<?, ?it/s]

[INFO 37:14] - learning_users_emb: 0.5
[INFO 37:15] rmse : 0.5669613480567932
[INFO 37:15] valid_rmse : 3.4154298305511475
[INFO 37:15] valid_loss : 6.258544445037842


  1%|          | 1/100 [00:02<04:36,  2.79s/it]

[INFO 37:17] - learning_users_emb: 0.5
[INFO 37:18] rmse : 0.5090577602386475
[INFO 37:18] valid_rmse : 3.066612958908081
[INFO 37:18] valid_loss : 4.702308654785156


...

[INFO 39:49] - learning_users_emb: 0.0078125
[INFO 39:50] rmse : 0.44190770387649536
[INFO 39:50] valid_rmse : 2.6620945930480957
[INFO 39:50] valid_loss : 3.595012903213501


 58%|█████▊    | 58/100 [02:37<01:54,  2.72s/it]

[INFO 39:50] -- END Training --
[INFO 39:50] train on cuda



100%|██████████| 6/6 [00:00<00:00,  8.92it/s]


-0.022887408128349388
({0: {'mi_acc': 0.7017393112182617}, 1: {'mi_acc': 0.7017393112182617}, 2: {'mi_acc': 0.7017393112182617}, 3: {'mi_acc': 0.7017393112182617}, 4: {'mi_acc': 0.7017393112182617}, 5: {'mi_acc': 0.7017393112182617}}, {0: {'meta_doa': np.float64(0.4731912698967316)}, 1: {'meta_doa': np.float64(0.4785325609134498)}, 2: {'meta_doa': np.float64(0.488984180994137)}, 3: {'meta_doa': np.float64(0.4829888078191557)}, 4: {'meta_doa': np.float64(0.4831064777484556)}, 5: {'meta_doa': np.float64(0.4797462898645038)}})


In [ ]:
del S

In [ ]:
meta_trainers = ['GAP']
for meta_trainer in meta_trainers : 
    config['meta_trainer'] = meta_trainer
    logging.info(f'#### meta_trainer : {config["meta_trainer"]} ####')
    for i_fold in range(1,2) : 
        config['i_fold'] = i_fold
            
        logging.info(f'#### i_fold : {i_fold} ####')
        ## Dataframe columns : (user_id, question_id, response, category_id)
        train_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_train_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})
        valid_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_valid_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})
        test_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_test_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})

        train_data = liriscat.dataset.CATDataset(train_df, concept_map, metadata, config,nb_modalities)
        valid_data = liriscat.dataset.EvalDataset(valid_df, concept_map, metadata, config,nb_modalities)
        test_data = liriscat.dataset.EvalDataset(test_df, concept_map, metadata, config,nb_modalities)

        for seed in range(1) :
            config['seed'] = seed
            logging.info(f'#### seed : {seed} ####')

            train_data.reset_rng()
            valid_data.reset_rng()
            test_data.reset_rng()

            S = liriscat.selectionStrategy.Random(metadata,**config)
            S.init_models(train_data, valid_data)
            S.train(train_data, valid_data)
            liriscat.utils.set_seed(0)
            S.reset_rng()
            d = (S.evaluate_test(test_data, train_data, valid_data))
            logging.info(d)
            logging.info(liriscat.utils.pareto_index(d))

    torch.cuda.empty_cache()

In [ ]:
config['meta_trainer'] = 'GAP'
#config['meta_trainer'] = 'Adam'
for seed in range(1) :
    config['seed'] = seed
    logging.info(f'#### seed : {seed} ####')
    
    train_data.reset_rng()
    valid_data.reset_rng()
    test_data.reset_rng()


    S = liriscat.selectionStrategy.Random(metadata,**config)
    S.init_models(train_data, valid_data)
    S.train(train_data, valid_data)
    liriscat.utils.set_seed(0)
    S.reset_rng()
    d = (S.evaluate_test(test_data, train_data, valid_data))
    print(liriscat.utils.pareto_index(d))
    print(d)
    torch.cuda.empty_cache()